# Aerated Bioreactor Design for Oleochemical Production

The oleochemical industry is focusing renewable feedstocks and more sustainable pathways of production. Microbial pathways are generally not market competitive for the production of commodity oleochemicals, but technological breakthroughs in metabolic engineering are making microbial pathways a promising platform. In this case study, we discuss how to model an aerobic fermentation system for producing microbial oil, which can be further upgraded to sustainable aviation fuel or other oleochemicals. The aerobic fermentation system (including the compressor, reactor, agitator, and heat exchange equipment) constitutes the major cost driver in the minimum oil selling price.

The goals of this case study are to:
* Understand the design basis of sizing an aerated bioreactor in BioSTEAM.
* Put in practice BioSTEAM's design algorithms for microbial oil production.
* Leverage senstivity analysis for uncovering key economic drivers. 

## Design basis

### Configuration

In this case study, we implement an industry-relevant configuration consisting of a stirred tank reactor, a heat exchanger loop, and a compressor (Figure 1). If you feedstock is in the form of diluted sugars (e.g., lignocellusic hydrolysate), you may need to concentrated your feed to achieve the a specified titer. This can be done efficiently through multi-effect evaporation, whereby the steam from the first effect is used to boil the next effect using a vacuum system. Alternatively, if you wish to operate at lower titers, you can dilute the sugars. Here we include both possibilities to be able to evaluate a spectrum of titers. Due to the high risk of bacterial contamination, we choose to operate in batch mode. Fed-batch configurations are also possible and can be modeled in a similar fashion. 

![Aerated bioreactor configuration](aerated_bioreactor_configuration.svg)

### Mass transfer and mixing

Aerobic processes demand an oxygen uptake rate (OUR) which is satisfied by the oxygen transfer rate (OTR) under quasi steady-state. The basic mass tranfer equation is:

$$ OTR = k_La(C_{out} - C) $$

For tall vessels (>1 m tall) the log-mean driving force is more accurate. Due to the oxygen partial pressure gradient, the local concentration and the saturation concentration are different at the top and bottom of the reactor:

$$ OTR = k_La \frac{(C_{sat} - C)_{out} - (C_{sat} - C)_{in}}{ln\Big(\large \frac{(C_{sat} - C)_{out}}{(C_{sat} - C)_{in}} \Big)} $$

The $k_La$ can be correlated to the superficail gas velocity, $U_S$, and the agitator power per unit volume, $P$, as follows:

$$ k_La = AP^BU_S^C $$

Where A, B, and C are correlation coefficients dependent on the agitator system, broth type, and size dimensions. The detailed design of the agitator is not relevant at this stage of the analysis, but the correlation coefficients will be a function of the agitator design.

**Assumptions**

* Assume a dissolved oxygen concentration of 50% saturation. Note that the flow rate of air (or agitation power) required to achieve an oxygen uptake rate equal to the oxygen transfer rate will depend on the oxygen saturation.
* Set the power consumed by the agitator to 0.985 kW∙m3, a heuristic value common for industrial homogeneous reactions and comparable to estimates in Aspen plus. Note that is also possible to minimize power consumption by varying flow rate, but this would also lead to a greater capital cost for the compressor.
* Assume the Van't Riet’s non-viscous mass transfer correlation is applicable to estimate the overall mass transfer coefficient, kLa (lumped together with the specific interfacial area). 

In [1]:
theta_O2 = 0.5 # Dissolved oxygen concentration [% saturation]
agitation_power = 0.985 # [kW / m3]
kLa_coefficients = "Van't Riet" # Name of method

### Heat transfer

The heat exchanger loop provides additional mixing with the trade-off of temperature gradients throughout the loop. The fluid at the inlet of the heat exchanger is at the reactor operating temperature while the outlet fluid will be at a lower temperature, which depends on the flow rate. We can specify the outlet temperature (which cannot be too low to avoid inactivation of the yeast) and solve for the recirculation flow rate:

$$F = \frac{Duty}{C_pT_{outlet} - C_pT_{inlet}} $$

The detailed design of the heat exchanger follows by-the-book design and consting algorithms, but is not discussed here.

**Assumptions**

* The fermentation temperature is 32 $^\circ$C
* Assume the temperature of the heat exchanger outlet does not drop by more than 8 $^\circ$C
* Assume the aerobic fermenter has a high heat-production rate of 110 kcal per mol of O2 consumed.

In [2]:
T_operation = 273.15 + 32 # [K]
Q_O2_consumption = -110 * 4184 # [kJ/kmol]
dT_hx_loop = 8 # degC

## Compressor

The compressor is needed to pressurize air to make up for the liquid head and losses from the sparger ring and piping. Currently, BioSTEAM only accounts for the liquid head. The outlet from the compressor has an increased temperature due to thermodynamic inefficiencies. For this reason, a cooler is also needed to brin the temperature back down.

### Reaction stoichiometry

The production of microbial oil is an aerobic process that can be modeled as the sum of 3 stoichiometric reactions: oil production from sugar, combustion of glucose (respiration), and cell growth. The substrate not converted to cell mass or oil is assumed to be consumed for respiration. Given the yield of triolein, Yp (wt %), and the fraction of biomass produced over the total substrate not consumed for oil production, Yb (wt %,), the extent of each reaction can be calculated. 

| Reaction            | Stoichiometry (by mol)                                             | Reaction extent               |
| ------------------- | ------------------------------------------------------------------- | ----------------------------- |
| Production:         | $$ Glucose \rightarrow 0.235\ H_2O + 2.53\ O_2  + 0.118\ Tripalmitin $$ | $$ X_p=\frac{Y_p}{0.527}   $$ |
| <sup>a</sup>Growth: | $$ Glucose \rightarrow 1.7\ H_2O + 0.655\ CO_2 + 5.35\ Yeast      $$ | $$ X_b= (1-X_p ) \cdot Y_b $$ |
| Respiration:        | $$ Glucose + 6 O_2 \rightarrow 6 H_2O + 6 CO_2                    $$ | $$ X_r=1-X_p-X_b           $$ | 

<center><sup>a</sup>The molecular formula of yeast is assumed to be $CH_{1.61}O_{0.56}$</center>

**Assumptions**

* The maximum volume of each reactor vessel is 200 $\text{m}^\text{3}$.
* Titer is 27.4 g∙L<sup>-1</sup>
* Productivity is 0.31 g∙L<sup>-1</sup>∙h<sup>-1</sup> 
* Yield is 18 wt %
* Ratio of biomass produced over total substrate not consumed for oil production is 43 wt %

In [3]:
V_max = 200 # [m3]
titer = 27.4 # [g / L]
productivity = 0.31 # [g / L / h]
lipid_yield = 0.18 # [by wt]
Y_b = 0.43 # [by wt]

## Bioreactor modeling in BioSTEAM

With an introductory understanding of the design principles, we can now model our configuration. The first step we will take is to create the chemicals and formulate the reactions:

In [29]:
import biosteam as bst
bst.nbtutorial()
bst.settings.set_thermo([
    bst.Chemical('Glucose', phase='l'),
    bst.Chemical('Water'),
    bst.Chemical('CO2'),
    bst.Chemical('O2'),
    bst.Chemical('N2'),
    bst.Chemical('Tripalmitin', phase='l', Hf=-2468.7),
    bst.Chemical('Yeast', 
        phase='s',
        search_db=False,
        formula='CH1.61O0.56',
        rho=1540, # kg/m3
        Cp=1.5, # J/g
        default=True, # Default other properties
    )
])
chemicals = bst.settings.chemicals
chemicals.set_alias('Tripalmitin', 'Lipid')
chemicals.Lipid.V.method = 'HTCOSTALD'
production = bst.Reaction(
    "Glucose -> H2O + O2 + Lipid", reactant='Glucose', X=1,
    correct_atomic_balance=True,
)
production.product_yield('Lipid', basis='wt', product_yield=lipid_yield)
growth = bst.Reaction(
    'Glucose -> H2O + CO2 + Yeast', 'Glucose', (1 - lipid_yield) * Y_b, 
    correct_atomic_balance=True
)
respiration = bst.Reaction(
    'Glucose + O2 -> CO2 + H2O', 'Glucose', 1. - growth.X - production.X,
    correct_atomic_balance=True
)
reactions = bst.ParallelReaction([production, growth, respiration])
reactions.show()

ParallelReaction (by mol):
index  stoichiometry                                         reactant    X[%]
[0]    Glucose -> 0.235 Water + 2.53 O2 + 0.118 Tripalmitin  Glucose    34.14
[1]    Glucose -> 1.7 Water + 0.655 CO2 + 5.35 Yeast         Glucose    35.26
[2]    Glucose + 6 O2 -> 6 Water + 6 CO2                     Glucose    30.60


Now let's create our aerated bioreactor:

In [30]:
effluent = bst.Stream('effluent')
AB1 = bst.AeratedBioreactor('AB1',
    ins=['feed', bst.Stream('air', phase='g')],
    outs=['vent', effluent],
    V_max=V_max, Q_O2_consumption=Q_O2_consumption,
    dT_hx_loop=dT_hx_loop, T=T_operation,
    batch=True, reactions=reactions,
    kW_per_m3=agitation_power,
    tau=titer/productivity,
)

performance = {
    'titer': titer,
    'productivity': productivity,
    'yield': lipid_yield,
}

@AB1.add_specification(run=True)
def update_reaction_time():
    AB1.tau = performance['titer'] / performance['productivity']

AB1.diagram()

The bioreactor system already comes with all the auxiliary units for aeration and heat transfer. All what is left is to add the concentration/dilution system. We create the multi-effect evaporator assuming 5 effects. The required vapor fraction of the first effect will be solved for in a specification:

In [34]:
from flexsolve import wegstein as itersolve

feedstock = bst.Stream('feedstock',
    Water=0.85, Glucose=0.15, 
    total_flow=1e4, units='kg/hr'
)
E1 = bst.MultiEffectEvaporator('E1',
    ins=feedstock, outs=('evaporated_feed', 'condensate'), 
    V=0, V_definition='First-effect',
    P=(101325, 73581, 50892, 32777, 20000),
)
dilution_water = bst.Stream('dilution_water')
M1 = bst.Mixer('M1',
    ins=(E1-0, dilution_water), outs=AB1.ins[0]
)
simulation_path = E1.path_until(AB1, inclusive=True)

def get_titer(): # g/L or kg/m3
    product_mass_flow = effluent.imass['Lipid'] # effluent.get_flow('kg / hr', 'Lipid')
    volumetric_flow_rate = effluent.F_vol # effluent.get_total_flow('m3/hr')
    return product_mass_flow / volumetric_flow_rate

def adjust_dilution_water(water):
    dilution_water.imass['Water'] = water
    for unit in simulation_path: unit._run()
    target = performance['titer']
    current = get_titer()
    rho = chemicals.Water.rho('l', T=T_operation, P=101325) # kg / m3
    return water + (1./target - 1./current) * effluent.imass['Lipid'] * rho
    
@E1.add_bounded_numerical_specification(x0=0, x1=0.15, ytol=1e-3, xtol=1e-6, maxiter=20)
def evaporation(V):
    E1.V = V
    if V == 0: 
        needed_water = itersolve(adjust_dilution_water, 0, xtol=1e-3)
        if needed_water > 0: return performance['titer'] - get_titer()
    dilution_water.empty()
    for unit in simulation_path: unit._run()
    return performance['titer'] - get_titer()

microbial_lipids_sys = bst.main_flowsheet.create_system('microbial_lipids_sys')
microbial_lipids_sys.simulate()
microbial_lipids_sys.diagram(auxiliaries=True)

In [33]:
AB1.results()

Aerated bioreactor                                         Units                  AB1
Electricity         Power                                     kW             2.45e+04
                    Cost                                  USD/hr             1.92e+03
Chilled water       Duty                                   kJ/hr            -4.64e+06
                    Flow                                 kmol/hr              3.1e+03
                    Cost                                  USD/hr                 23.2
Design              Reactor volume                            m3                  192
                    Batch time                                                    107
                    Loading time                                                 15.2
                    Residence time                            hr                 88.4
                    Vessel type                                              Vertical
                    Length                                    ft                 42.7
                    Diameter                                  ft                 14.2
                    Weight                                    lb             4.49e+04
                    Wall thickness                            in                 0.48
                    Vessel material                               Stainless steel 316
Purchase cost       Vertical pressure vessel (x7)            USD             1.96e+06
                    Platform and ladders (x7)                USD             3.11e+05
                    Compressor - Compressor(s)               USD             3.07e+05
                    Air cooler - Double pipe                 USD             5.28e+03
                    Heat exchanger - Floating head (x7)      USD             1.55e+05
                    Recirculation pump - Pump (x7)           USD             3.08e+04
                    Recirculation pump - Motor (x7)          USD             3.79e+03
                    Agitator - Agitator (x7)                 USD             3.78e+06
Total purchase cost                                          USD             6.55e+06
Utility cost                                              USD/hr             1.94e+03

Now that we have a working model, let's leverage it to evaluate a landscape of potential fermentation scenarios:

In [ ]:
productivity = np.array([
    0.5 * productivity,
    productivity,
    2 * productivity,
])
X, Y, Z = generate_contour_data(
    evaluate_titer_yield_productivity,
    xlim=[0.5 * titer, 1.5 * titer],
    ylim=[0.5 * lipid_yield, 1.5 * lipid_yield],
    args=(
        productivity,
    ),
    file='fermentation_performance_data',
    load=load,
    n=10,
)

# Plot contours
xlabel = 'Microbial oil recovery [%]'
ylabel = "Microb. oil yield [wt. %]"
units = r'$g \cdot L^{-1} \cdot h^{-1}$'
ylabels = [f"{ylabel}\nat {round(i, 2)} {units}" for i in productivity]
xticks = [50, 60, 70, 80, 90]
yticks = rounded_linspace(
    100 * perf.min_lipid_yield_glucose, 100 * perf.max_lipid_yield_glucose, 5, 
)
titles = ['Mechanical oil recovery', 'Integrated oil recovery']
metric_bar = MetricBar(
    metric.name, format_units(metric.units), plt.cm.get_cmap('viridis_r'), tickmarks(Z[~np.isnan(Z)], 5, 1, expand=0, p=0.5), 15, 1
)
fig, axes, CSs, CB, other_axes = plot_contour_single_metric(
    X, Y, Z, xlabel, ylabels, xticks, yticks, metric_bar,  titles,
    fillcolor=None, styleaxiskw=dict(xtick0=False), label=True,
)
return fig, other_axes

In [ ]:
[i for i in chemicals.Lipid.V.l.ranked_methods if i in chemicals.Lipid.V.l.all_methods]


In [ ]:
chemicals.Lipid.V.l.plot_T_dependent_property()